In [1]:
from dotenv import load_dotenv
import os
import pandas as pd

from langchain.vectorstores import Chroma
from langchain_community.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

/home/moonyoung/anaconda3/envs/gemini-langchain/lib/python3.13/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)
/home/moonyoung/anaconda3/envs/gemini-langchain/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv('../')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_output_tokens=200,
    google_api_key=GEMINI_API_KEY
)

In [4]:
df = pd.read_excel("./maum_ai_product_sample.xlsx")
loader = DataFrameLoader(df, page_content_column="설명")
docs = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)

In [6]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07", google_api_key=GEMINI_API_KEY)
vectorstore = Chroma.from_documents(chunks, embeddings)

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [8]:
contextualize_q_system_prompt = """
    Given a chat history and the latest user question which might reference context in the chat history,
    formulate a standalone question which can be understood without the chat history.
    Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [9]:
class ResponseGenerator():
    def __init__(self):
        self.template = ""
        self.ragchain = self.rag_chain_generator()
        self.store = {}

    def template_generator(self):
        template = """
                당신은 이 회사의 상품을 추천하는 임부를 부여받았습니다.
                고객들은 당신에게 AI가 필요한 특정 상황을 설명할 것입니다.
                당신은 고객 문의에 대해 단계별로 차근차근 친절하게 설명해 주세요.
                없는 정보는 답하면 안 됩니다.
                반드시 회사 상품 하나를 추천해야 합니다.
                끝에는 언제나 '감사합니다!'라는 멘트를 붙여주세요.
                \n\n
                {context}
                질문: {input}
                You MUST answer in Korean:
                """
        return template
    
    def prompt_generator(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system", self.template_generator()),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}")
        ])
        return prompt
    
    def rag_chain_generator(self):
        question_answer_chain = create_stuff_documents_chain(
            llm, self.prompt_generator()
        )

        history_aware_retriever = create_history_aware_retriever(
            llm, retriever, contextualize_q_prompt
        )

        rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
        
        return RunnableWithMessageHistory(
            rag_chain,
            self.get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer"
            )
    
    def get_session_history(self, session_ids):
        if session_ids not in self.store:  
            self.store[session_ids] = ChatMessageHistory()
        return self.store[session_ids]  

In [10]:
rg = ResponseGenerator()

In [11]:
def response_from_llm(text, session_id):
    rag_chain = rg.ragchain
    
    result = rag_chain.invoke(
        {"input": text}, 
            config = {"configurable": {"session_id": session_id}}
    )["answer"]

    return result

In [12]:
response_from_llm('자율주행 ai에 대해 문의하고 싶어요.', 1)

'안녕하세요! 자율주행 AI에 대해 문의하셨군요.  자세히 말씀해주시면 제가 도와드리겠습니다. 어떤 부분에 대해 궁금하신가요? 예를 들어, 자율주행 AI의 작동 원리,  적용 분야,  구축 방법 등 구체적으로 질문해주시면 더욱 정확한 답변을 드릴 수 있습니다.\n\n하지만 혹시 시각 정보 기반의 End-to-End 자율주행에 관심이 있으신가요?  그렇다면 저희 회사의 **E2E Autonomous Driving Kit**를 추천드립니다.\n\n이 키트는 다음과 같은 장점을 가지고 있습니다.\n\n1. **시각 정보 기반:** 카메라 등 시각 정보만을 이용하여 주변 환경을 인식하고'